# Scraping articles by Enrique Krauze

In [69]:
import time
import re

import pandas as pd
import numpy as np

import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

## Get links of all articles

In [73]:
# Configure driver
profile = webdriver.FirefoxProfile()
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:86.0) Gecko/20100101 Firefox/86.0'
profile.set_preference("general.useragent.override", user_agent)
options = Options()
options.headless = True

# Launch driver
driver = webdriver.Firefox(profile, options=options)
#driver.maximize_window()
driver.get('https://enriquekrauze.com.mx/category/articulos/')

# Load complete list of articles
while True: 
    try: 
        #Wait for the button "Cargar más" to load
        element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, '__k-section-button')))
        
        #Click on "Cargar más" button
        driver.find_element_by_class_name('__k-section-button').click()
    
    except:
        print('No more "Cargar más" buttons to click')
        break
        
# Get links of articles
pageSource = driver.page_source
soup = BeautifulSoup(pageSource, 'html.parser')
links = []
for link in soup.find_all('h3', {'class': '__news-grid-title'}):
    x = link.a['href']
    links.append(x)

No more "Cargar más" buttons to click


## Extract content of each article

In [75]:
def krauze_content_scrapper(link):
    
    """Get title, date, and body of an article of Enrique Krauze"""
    
    # Get text of website
    html = urlopen(link)
    soup = BeautifulSoup(html, 'lxml')
    
    # Get title
    title = soup.find('h3', {'class': 'kz-single-post__title'}).get_text()
    
    # Get date
    date = soup.find('div', {'class': 'kz-single-post__date'}).get_text()
    
    # Get body of article
    body = soup.find_all('p')[1:] 
    
    # Return
    return title, date, body

In [76]:
# Create variables to store information
data_krauze = {'title': '', 'date': '', 'body': ''}
titles = []
dates = []
bodies = []

In [77]:
# Store information
for link in links:
    title, date, body = krauze_content_scrapper(link)
    time.sleep(1)
    titles.append(title)
    dates.append(date)
    bodies.append(body)

# Transform to DataFrame

In [78]:
# Format information
data_krauze['title'] = titles
data_krauze['date'] = dates
data_krauze['body'] = bodies

In [81]:
data = pd.DataFrame(data_krauze)
data.head(10)

,title,date,body
0,El jurista bondadoso,08 marzo 2021,"[[ wp:paragraph ], [La justicia es ciega, llev..."
1,La hambruna recordada,22 febrero 2021,"[[ wp:paragraph ], [Toda revolución debe verse..."
2,Una historia de Covid,08 febrero 2021,"[[ wp:paragraph ], [Martha creció con su numer..."
3,El espejo de Weimar,25 enero 2021,"[[ wp:paragraph ], [Toda democracia en el sigl..."
4,Contra el despotismo,11 enero 2021,"[[ wp:paragraph ], [Ningún país del mundo, ni ..."
5,El jardín de Adolfo Castañón,28 diciembre 2020,"[[ wp:paragraph ], [Una de las buenas noticias..."
6,Memoria de la democracia,14 diciembre 2020,"[[ wp:paragraph ], [Los únicos héroes de la de..."
7,Estadísticas y tragedia,30 noviembre 2020,"[[ wp:paragraph ], [""Un muerto es una tragedia..."
8,China: redentores prácticos,17 noviembre 2020,"[[ wp:paragraph ], [La reciente publicación en..."
9,¿Quién contará su historia?,05 octubre 2020,"[[ wp:paragraph ], [Setenta y ocho mil ochocie..."


In [82]:
data.tail(10)

,title,date,body
572,Santo Tomás en México,12 mayo 1991,"[[En un viaje reciente a Madrid, unos amigos e..."
573,TLC: prescripción contra la miopía,05 mayo 1991,[[Los norteamericanos son rápidos para reaccio...
574,Viajes que ilustran,14 abril 1991,[[Espero que los generosos lectores no crean q...
575,Postal del Japón,31 marzo 1991,[[Todo mundo tiene derecho a ganarse la loterí...
576,Havel recuerda Munich,17 febrero 1991,[[Checoslovaquia es uno de los países que part...
577,Ideas impopulares,03 febrero 1991,[[[No hay argumentos contra un estado de ánimo...
578,Manuel J. Clouthier: Coraje cívico,03 octubre 1989,"[[Creía en el martirio, por eso fue tan doloro..."
579,Desmesura,01 noviembre 1984,"[[De 1972 a 1976, el Fondo de Cultura Económic..."
580,Ecos de la renuncia de Fuentes,01 junio 1977,"[[Aparte de su propio mérito moral, la renunci..."
581,Cosío Villegas y Excélsior,01 diciembre 1976,[[Mucho tiempo antes de que ocurriera el coup ...


In [83]:
data['source'] = 'Personal website'
data['link'] = links
data.insert(0, 'author', 'Enrique Krauze')
data.head()

,author,title,date,body,source,link
0,Enrique Krauze,El jurista bondadoso,08 marzo 2021,"[[ wp:paragraph ], [La justicia es ciega, llev...",Personal website,https://enriquekrauze.com.mx/el-jurista-bondad...
1,Enrique Krauze,La hambruna recordada,22 febrero 2021,"[[ wp:paragraph ], [Toda revolución debe verse...",Personal website,https://enriquekrauze.com.mx/la-hambruna-recor...
2,Enrique Krauze,Una historia de Covid,08 febrero 2021,"[[ wp:paragraph ], [Martha creció con su numer...",Personal website,https://enriquekrauze.com.mx/una-historia-de-c...
3,Enrique Krauze,El espejo de Weimar,25 enero 2021,"[[ wp:paragraph ], [Toda democracia en el sigl...",Personal website,https://enriquekrauze.com.mx/el-espejo-de-weimar/
4,Enrique Krauze,Contra el despotismo,11 enero 2021,"[[ wp:paragraph ], [Ningún país del mundo, ni ...",Personal website,https://enriquekrauze.com.mx/contra-el-despoti...


# Save as .csv

In [84]:
# Save as csv
data.to_csv('../Data/data_Enrique_Krauze', index=False)